In [17]:
%load_ext autoreload
%autoreload 2

# Introduction

## Setup

In [18]:
!jupyter nbextension enable --py widgetsnbextension

In [19]:
import pandas as pd
from urllib import request
from tqdm.notebook import tqdm
from pathlib import Path
import yaml

## Download metadata file from Encode

In [20]:
!wget -O metadata.tsv "https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released"

--2024-06-11 15:38:21--  https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tsv]
Saving to: ‘metadata.tsv’

metadata.tsv            [   <=>              ] 584.85K   763KB/s    in 0.8s    

2024-06-11 15:38:23 (763 KB/s) - ‘metadata.tsv’ saved [598890]



In [21]:
metadata = pd.read_csv('metadata.tsv', sep='\t')

In [22]:
metadata

,File accession,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Donor(s),Biosample term id,Biosample term name,Biosample type,Biosample organism,Biosample treatments,Biosample treatments amount,Biosample treatments duration,Biosample genetic modifications methods,Biosample genetic modifications categories,Biosample genetic modifications targets,Biosample genetic modifications gene targets,Biosample genetic modifications site coordinates,Biosample genetic modifications zygosity,Experiment target,Library made from,Library depleted in,Library extraction method,Library lysis method,Library crosslinking method,Library strand specific,Experiment date released,Project,RBNS protein concentration,Library fragmentation method,Library size range,Biological replicate(s),Technical replicate(s),Read length,Mapped read length,Run type,Paired end,Paired with,Index of,Derived from,Size,Lab,md5sum,dbxrefs,File download URL,Genome annotation,Platform,Controlled by,File Status,s3_uri,Azure URL,File analysis title,File analysis status,Audit WARNING,Audit NOT_COMPLIANT,Audit ERROR
0,ENCFF053PAM,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR521GON,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,K562,cell line,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXOSC10-human,RNA,NaN,NaN,NaN,ultraviolet irradiation,reverse,2022-11-16,ENCODE,NaN,see document,175-300,1,1_1,NaN,NaN,NaN,NaN,NaN,NaN,"/files/ENCFF338WDB/, /files/ENCFF020RMT/",1550073,"Gene Yeo, UCSD",30f83e561ef7cf74e2715c78caa0525b,NaN,https://www.encodeproject.org/files/ENCFF053PA...,V29,NaN,NaN,released,s3://encode-public/2022/01/12/ddc40387-9584-48...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
1,ENCFF448LBY,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR521GON,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,K562,cell line,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXOSC10-human,RNA,NaN,NaN,NaN,ultraviolet irradiation,reverse,2022-11-16,ENCODE,NaN,see document,175-300,2,2_1,NaN,NaN,NaN,NaN,NaN,NaN,"/files/ENCFF267USI/, /files/ENCFF650HUV/",948611,"Gene Yeo, UCSD",99465bcb457cc7ae03b4e5d88989f26e,NaN,https://www.encodeproject.org/files/ENCFF448LB...,V29,NaN,NaN,released,s3://encode-public/2022/01/12/aba6b59b-6278-4b...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
2,ENCFF663WYH,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR521GON,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,K562,cell line,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXOSC10-human,RNA,NaN,NaN,NaN,ultraviolet irradiation,reverse,2022-11-16,ENCODE,NaN,see document,175-300,"1, 2","1_1, 2_1",NaN,NaN,NaN,NaN,NaN,NaN,"/files/ENCFF338WDB/, /files/ENCFF267USI/, /fil...",28276,"Gene Yeo, UCSD",cfd0130150b890234f0444280f59b817,NaN,https://www.encodeproject.org/files/ENCFF663WY...,V29,NaN,NaN,released,s3://encode-public/2022/01/12/db0f7f4a-8a13-4b...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
3,ENCFF332VUJ,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR391BBA,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,K562,cell line,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIF4E-human,RNA,NaN,NaN,NaN,ultraviolet irradiation,reverse,2022-07-19,ENCODE,NaN,see document,175-300,2,2_1,NaN,NaN,NaN,NaN,NaN,NaN,"/files/ENCFF748VHF/, /files/ENCFF568VFK/",846426,"Gene Yeo, UCSD",cd2f6020f4ed3538c6729cdf21cf5959,NaN,https://www.encodeproject.org/files/ENCFF332VU...,V29,NaN,NaN,released,s3://encode-public/2022/01/12/d16028dd-a9a0-47...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
4,ENCFF412VGD,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR391BBA,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,K562,cell line,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIF4E-human,RNA,NaN,NaN,NaN,ultraviolet irradiation,reverse,2022-07-19,ENCODE,NaN,see document,175-300,1,1_1,NaN,NaN,NaN,NaN,NaN,NaN,"/files/ENCFF967PSC/, /fil

## Prepare directory structure

In [23]:
BASE_FILE_PATH = Path("./csv/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)
    
BASE_FILE_PATH.mkdir()

In [24]:
!rm metadata.tsv

## Transform data

In [25]:
# get all unique protein names - all of our classes
targets = metadata['Experiment target'].unique()
targets

array(['EXOSC10-human', 'EIF4E-human', 'SRSF9-human', 'FMR1-human',
       'SRSF1-human', 'EEF2-human', 'TARDBP-human', 'PABPC4-human',
       'MORC2-human', 'RNF187-human', 'FXR2-human', 'GARS-human',
       'FXR1-human', 'MBNL1-human', 'APEX1-human', 'RPS10-human',
       'ADAT1-human', 'WRN-human', 'AQR-human', 'NSUN2-human',
       'SSB-human', 'ABCF1-human', 'FTO-human', 'EXOSC5-human',
       'DDX51-human', 'PCBP1-human', 'PHF6-human', 'WDR43-human',
       'NIP7-human', 'PUM1-human', 'UTP18-human', 'PRPF4-human',
       'ZNF800-human', 'SDAD1-human', 'SAFB-human', 'KHSRP-human',
       'UTP3-human', 'AATF-human', 'APOBEC3C-human', 'TIAL1-human',
       'GRWD1-human', 'WDR3-human', 'NIPBL-human', 'MATR3-human',
       'AKAP1-human', 'DDX21-human', 'YBX3-human', 'HLTF-human',
       'SUPV3L1-human', 'FUS-human', 'UPF1-human', 'PABPN1-human',
       'DDX52-human', 'CPEB4-human', 'ZC3H8-human', 'G3BP1-human',
       'EIF3G-human', 'BCLAF1-human', 'HNRNPC-human', 'STAU2-human',
     

This step takes about 50 minutes on a single core machine.

In [34]:
BED_HEADER = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
ALLOWED_CHROMOSOMES = ['chr' + str(i) for i in range(1, 22)] + ['chrX', 'chrY', 'chrMT']

coord_links = []

for target in tqdm(targets):

  # iterate over all experiments for current protein
  for experiment in metadata[metadata['Experiment target'] == target]['Experiment accession'].unique():
    
    protein_name = target[: target.index('-')]
    rows = metadata[(metadata['Experiment accession'] == experiment) & (metadata['Experiment target'] == target)]

    if (len(rows) > 1):
      # we are in a situation with multiple replicates

      # look at values in column Biological replicate(s) and choose the one where replicate numbers are merged
      rows = rows[rows['Biological replicate(s)'].str.contains(',')]  # keep only rows with multiple replicates

    # In general, it is not a good practice to create a DataFrame in loop by appending. 
    # But since we know that we have only one or two rows for these targets, we can use it.
    df = pd.DataFrame([], columns=['chr', 'start', 'end', 'strand'])
    for index, row in rows.iterrows():

      local_file = row['File accession'] + '.bed.gz'

      # downloading files in while-try, because network connection might break sometimes
      success = False
      while not success:
        try:
          request.urlretrieve(row['File download URL'], local_file)
          success = True
        except:
          print('Problem with file ', local_file, '. Trying again.')
      coord_links.append(row['File download URL'])
      reads = pd.read_csv(local_file, sep='\t', compression='gzip', header=None, names=BED_HEADER)
      
      # keep just necessary columns
      reads = reads[['chrom', 'chromStart', 'chromEnd', 'strand']]
      # keep just sequences from chromosomes chr1 - ch22, chrX, chrX and chrMT
      reads = reads[reads['chrom'].isin(ALLOWED_CHROMOSOMES)]
      # removing outliers - keep just sequences >= 20 bp and <= 100 bp
      reads = reads[((reads['chromEnd'] - reads['chromStart']) >= 20) & ((reads['chromEnd'] - reads['chromStart']) <= 100)]
      # rename columns
      reads.columns = ['chr', 'start', 'end', 'strand']

      # add DataFrame from current file to DataFrame from previous file
      df = pd.concat([df, reads], ignore_index=True)
      
      # cleaning - delete downloaded bed file
      Path(local_file).unlink()

  # add column with name of protein
  df['protein_name'] = protein_name

  filename = protein_name + '.csv.gz'
  df.to_csv(BASE_FILE_PATH / filename, index=False, compression='gzip')

with open(BASE_FILE_PATH / 'coord_links.txt', 'w') as f:
  f.write(f'{protein_name}: {coord_links}\n')

  0%|          | 0/168 [00:00<?, ?it/s]

100%|██████████| 168/168 [07:56<00:00,  2.84s/it]


## YAML file

In [30]:
# YAML file with metadata
# we store paths of reference fasta files

desc = {
  'version': 0,
  'classes': {}
}
for target in targets:
  name = target[: target.index('-')]
  desc['classes'][name] = {
      'type': 'fa.gz',
      'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
      'extra_processing': 'ENSEMBL_HUMAN_GENOME'
  }

with open(BASE_FILE_PATH / 'metadata.yaml', 'w') as fw:    
    yaml.dump(desc, fw)

desc

{'version': 0,
 'classes': {'EXOSC10': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'EIF4E': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'SRSF9': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'FMR1': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'SRSF1': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'EEF2': {'type': 'fa.